In [1]:
!pip install -q --upgrade pip
!pip install -q gradio pinecone-client==3.0.0 sentence-transformers transformers pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 133.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 142.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import gradio as gr
from pinecone import Pinecone, ServerlessSpec

import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import pipeline

print("✅ All packages installed successfully!")


✅ All packages installed successfully!


In [6]:
from google.colab import files
uploaded = files.upload()


Saving Extended_TravelRecommendations.csv to Extended_TravelRecommendations (2).csv


In [7]:
# 🔴 Replace your Pinecone API Key below:
pc = Pinecone(api_key="pcsk_4ZpWLj_GBTJtgbjKr9TJXCn1PgYTpkBK55htQSoS3xjGFk7KjihDjVspFjiekFMnXBsVVM")
index_name = "travel-recommendation-index"

# Create index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name)

print("✅ Pinecone initialized and index ready!")

# =========================
# 3. Upload and Embed Dataset
# =========================
# 🔵 Upload CSV manually if not uploaded yet
from google.colab import files
uploaded = files.upload()

# 🔵 Load the CSV
travel_data = pd.read_csv("/content/Extended_TravelRecommendations.csv")

# 🔵 Load embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# 🔵 Create embeddings
travel_embeddings = embedding_model.encode(travel_data['travel_preferences'].tolist()).tolist()

# 🔵 Prepare vectors for Pinecone
vectors = [
    {"id": str(i), "values": emb, "metadata": {"destination": travel_data['recommended_destination'][i]}}
    for i, emb in enumerate(travel_embeddings)
]

# 🔵 Upsert (upload) into Pinecone
index.upsert(vectors=vectors)

print("✅ Dataset embedded and indexed into Pinecone successfully!")

# =========================
# 4. Load RAG Conversational Model
# =========================
rag_model = pipeline("text2text-generation", model="google/flan-t5-base")

print("✅ RAG conversational model loaded successfully!")

# =========================
# 5. Define Travel Recommendation Function
# =========================
def get_travel_recommendation(user_input):
    input_embedding = embedding_model.encode(user_input).tolist()
    results = index.query(vector=input_embedding, top_k=3, include_metadata=True)

    if results.matches:
        best_match = results.matches[0]
        destination = best_match.metadata.get('destination', 'Unknown Destination')
        confidence = best_match.score

        context = f"User preference: {user_input}"
        generated_response = rag_model(context, max_length=256)[0]['generated_text']

        return f"🏖️ **Recommended Destination:** {destination} (Confidence: {confidence:.2f})\n\n🧳 **Details:** {generated_response}"
    else:
        return "😔 Sorry, no matching destination found."

# =========================
# 6. Build and Launch Gradio App
# =========================
iface = gr.Interface(
    fn=get_travel_recommendation,
    inputs=gr.Textbox(lines=2, placeholder="Describe your travel preferences here..."),
    outputs="markdown",
    title="🏝️ Travel Recommendation Chatbot",
    description="Get personalized travel suggestions using AI! 🌎",
    theme="soft",
)

# ✅ Launch with share=True for Public URL
iface.launch(share=True)

✅ Pinecone initialized and index ready!


Saving Extended_TravelRecommendations.csv to Extended_TravelRecommendations (3).csv


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


✅ RAG conversational model loaded successfully!
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b4c19d43a99963bb1c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
